# 3.1 Sequence to Sequence translation (preprocessing steps)

In [202]:
import os, sys
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

BATCH_SIZE = 64
EPOCHS = 20
LSTM_NODES =256
NUM_SENTENCES = 20000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_SIZE = 50

## Question 1: Output sentence example
Read the input/output sentences from the available corpus and, following the tutorial, store this data in appropriate data structures. Consider the following hyper-parameter values which, unless otherwise stated, apply for the whole project.
```python
NUM_SENTENCES = 20000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
```
Note: the actual format of the parallel corpus provided here slightly differs from the one described in the tutorial. The best way to deal which such a difference is to marginally adapt the python instructions used to read the corpus. After reading the corpus, what is the content of `output_sentences[172]`?

In [203]:
def preprocess(resource):
    input_sentences = []
    output_sentences = []
    output_sentences_inputs = []

    count = 0
    for line in open(resource, encoding="utf-8"):
        count += 1
        if count > NUM_SENTENCES:
            break
        if '\t' not in line:
            continue
        input_sentence, output = line.rstrip().split('\t')[:2]
        output_sentence = output + ' <eos>'
        output_sentence_input = '<sos> ' + output
        input_sentences.append(input_sentence)
        output_sentences.append(output_sentence)
        output_sentences_inputs.append(output_sentence_input)
        
    return input_sentences, output_sentences, output_sentences_inputs

input_sentences, output_sentences, output_sentences_inputs = preprocess('resources/fra_eng.txt')
print('"' + output_sentences[172] + '"')

"Je suis touchée ! <eos>"


## Question 2: Input data size
Tokenize the input and output sentences.

Note: the actual tokenization depends on the exact methodology to filter out certain tokens or characters. For instance, punctuation marks could be filtered out or not. One sticks here to the process described in the tutorial and, as you should see, one uses a different strategy for tokenizing the input versus the output sentences. Please check the Keras Api and , in particular, the `filters` argument from the `Tokenizer` class. Report the actual input vocabulary size after tokenization, and the maximal number of tokens in an input sequence, by updating the default values listed below.

In [204]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index
max_input_len = max(len(sen) for sen in input_integer_seq)
print("(%d, %d)" % (len(word2idx_inputs), max_input_len))

(3511, 6)


## Question 3: Output data size

Report the actual output vocabulary size after tokenization, and the maximal number of tokens in an output sequence, by updating the default values listed below.

In [205]:
output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = output_tokenizer.word_index
num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("(%d, %d)" % (len(word2idx_outputs), max_out_len))
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')

(9523, 13)


## Question 4: Word Embedding

To limit a bit the computing resources needed to solve this project and to take benefit of a pre-trained word embedding, you will consider some glove word embedding as mentioned before. In particular, consider the following hyper-parameter value which, unless otherwise stated, applies to the whole project. You must consider the appropriate glove file, extracted from glove.6B.zip, accordingly.
```python
EMBEDDING_SIZE = 50
```
Report the last 3 coefficient values associated to the word `"hit"` in the appropriate row of the `embedding_matrix`, by updating the default values listed below. Use at least 3 significant digits.

In [206]:
from numpy import array
from numpy import asarray
from numpy import zeros

def embed(glove_file):
    embeddings_dictionary = dict()
    for line in glove_file:
        records = line.split()
        word = records[0]
        vector_dimensions = asarray(records[1:], dtype='float32')
        embeddings_dictionary[word] = vector_dimensions
    
    glove_file.close()

    num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
    embedding_matrix = zeros((num_words, EMBEDDING_SIZE))
    for word, index in word2idx_inputs.items():
        embedding_vector = embeddings_dictionary.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
    
    return embeddings_dictionary, embedding_matrix, num_words
    
glove_file = open('resources/glove.6B/glove.6B.50d.txt', encoding="utf8")
embeddings_dictionary, embedding_matrix, num_words = embed(glove_file)
print(tuple(embeddings_dictionary['hit'][-3:]))
embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=max_input_len)

(-0.23663, 0.29088, 0.11071)


## Question 5: One-hot target encoding

Target values need to be defined before training a neural translation model from input/output sequence pairs. This is implemented here as a multi-dimensional array with one-hot encoding of the target values. Report the number of dimensions of this array by updating the default values listed below.

Note: as you might experience, building up a neural machine translation system requires sufficient memory, disk usage, and quite a bit of computing resources. The proposed implementation of such a one-hot encoding is far from being efficient. You might notice for instance that the actual learning phase will be decomposed in consecutive mini-batches. Yet, the target values are predefined here for the whole corpus. Assuming your available memory and/or free disk space are sufficient, you can stick to such default implementation.

In [207]:
decoder_targets_one_hot = np.zeros((len(input_sentences),
                                    max_out_len,
                                    num_words_output),
                                   dtype='float32')
print(decoder_targets_one_hot.shape)

(20000, 13, 9524)


# 3.2 Sequence to Sequence translation (training)
You will now proceed with the actual training of a seq2seq architecture. This model follows here an encoder-decoder structure based on Long Short Term Memory cells (LSTMs). By default, you will consider all hyperparameter values specified in task 3.1 and the following ones:
```python
BATCH_SIZE = 64
LSTM_NODES = 256
VALIDATION_SPLIT = 0.1
```
This last hyper-parameter value means that, among the already loaded data, the last 10% (in terms of sentence number) will not be used for training but rather assessing a validation accuracy on independent data after each epoch of the training procedure. When compiling your model(s), make sure you use the same `optimizer`, `loss` and `metrics` as described in the tutorial.

## Question 1: Number of trainable parameters
After fixing all hyper-parameter values to their default values specified before, what is the exact number of trainable parameters in the model used for training.

Hint: Check the Keras `Model` class to look for an appropriate method to get information about a model already built.

In [208]:
VALIDATION_SPLIT = 0.1

for i, d in enumerate(decoder_output_sequences):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1
        
encoder_inputs_placeholder = Input(shape=(max_input_len,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

decoder_inputs_placeholder = Input(shape=(max_out_len,))

decoder_embedding = Embedding(num_words_output, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)

decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs_placeholder,
               decoder_inputs_placeholder],
              decoder_outputs)
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

from keras.utils.layer_utils import count_params

trainable_count = count_params(model.trainable_weights)
print(trainable_count)

5901092


## Question 2: Validation results after one epoch
Which are the loss and accuracy values after training the proposed model for a single epoch? Replace the default values by the correct ones. Use at least 2 significant digits.

In [82]:
r = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=BATCH_SIZE,
    epochs=1,
    validation_split=VALIDATION_SPLIT,
)

print('(%f, %f)' % (r.history['val_loss'][-1], r.history['val_accuracy'][-1]))

282/282 [==============================] - 264s 935ms/step - loss: 2.0300 - accuracy: 0.7196 - val_loss: 2.0780 - val_accuracy: 0.7060
(2.078035, 0.706000)


## Question 3: Validation results after 20 epochs
Which are the loss and accuracy values after training the proposed model for 20 epochs rather than a single one? Replace the default values below by the correct ones. Use at least 2 significant digits.

In [209]:
r = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1,
)

print('(%f, %f)' % (r.history['val_loss'][-1], r.history['val_accuracy'][-1]))

Epoch 1/20
282/282 [==============================] - 343s 1s/step - loss: 2.0373 - accuracy: 0.7173 - val_loss: 2.0651 - val_accuracy: 0.7062
Epoch 2/20
282/282 [==============================] - 366s 1s/step - loss: 1.5152 - accuracy: 0.7700 - val_loss: 1.8118 - val_accuracy: 0.7382
Epoch 3/20
282/282 [==============================] - 380s 1s/step - loss: 1.3067 - accuracy: 0.8027 - val_loss: 1.6865 - val_accuracy: 0.7555
Epoch 4/20
282/282 [==============================] - 250s 887ms/step - loss: 1.1857 - accuracy: 0.8185 - val_loss: 1.6174 - val_accuracy: 0.7643
Epoch 5/20
282/282 [==============================] - 240s 851ms/step - loss: 1.0976 - accuracy: 0.8291 - val_loss: 1.5648 - val_accuracy: 0.7694
Epoch 6/20
282/282 [==============================] - 238s 844ms/step - loss: 1.0255 - accuracy: 0.8380 - val_loss: 1.5268 - val_accuracy: 0.7774
Epoch 7/20
282/282 [==============================] - 234s 828ms/step - loss: 0.9683 - accuracy: 0.8452 - val_loss: 1.5105 - val_accu

## Question 4: Number of trained parameters of the encoder and of the decoder
After a careful reading of the tutorial, you might have noticed that the model used when training the seq2seq architecture is not the same as the one(s) used for making predictions, i.e. for actually translating a source sentence, presumably without knowing the target sentence. More specifically, how many trained parameters are included respectively in the encoder and the decoder used for prediction? Replace the default values below by the correct ones.

In [210]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)

decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs_single] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

enc_params = count_params(encoder_model.trainable_weights)
dec_params = count_params(decoder_model.trainable_weights)
print('(%d, %d)' % (enc_params, dec_params))

(489968, 5411124)


# 3.3 Making Predictions
You will now make actual predictions for translating a source (or input) sentence into a target (or output) sentence. For performing such predictions, it is assumed that you have already trained a neural machine translation model according to the default hyper-parameter values, summarized again below.
```python
NUM_SENTENCES = 20000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_SIZE = 50
BATCH_SIZE = 64
LSTM_NODES = 256
VALIDATION_SPLIT = 0.1
EPOCHS = 20
```
You might be tempted to call `model.predict(...)` to compute a prediction from your trained model, but the actual predictions must be computed from the combination of an encoder and a decoder derived from such a trained seq2seq architecture.

## Question 1: First translation example (training sentence)
What is the translation produced by your encoder-decoder model for the following input sentence?

    "I'm a lawyer."
    
### Notes
1. Recall that the actual inputs to the networks are made of one-hot encoding of the input words after tokenization, including the possible filtering of some punctuation marks, and zero-padding. For your prediction to make sense, the input sentence should go through the exact same process.
2. Your answer must respect the exact format of the output word sequence predicted by the model, including lower/upper case when it applies, punctuation mark(s) if any. The whole output sentence must be typed in between quotes (").

### Hint
To help you cross-check your implementation in this regard, this input sentence should be at the line 3853 in the parallel corpus. So, it has likely received a numerical index `i==3852` (since such an index usually starts at the `0` value) in some appropriate data structure(s). Note that the predicted translation is not guaranteed to be the correct translation according to this corpus.

In [211]:
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']
    eos = word2idx_outputs['<eos>']
    output_sentence = []
    
    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])
        
        if eos == idx:
            break
            
        word = ''
        
        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)
            
        target_seq[0, 0] = idx
        states_value = [h, c]
        
    return ' '.join(output_sentence)

In [256]:
i = 3852
input_seq = encoder_input_sequences[i:i+1]
translation = translate_sentence(input_seq)
print('"' + translation + '"')

"je suis avocat."


## Question 2: Second translation example (training sentence)
What is the translation produced by your encoder-decoder model for the following input sentence?

    "Is anybody hurt?"

Your answer must respect the exact format of the output word sequence predicted by the model, including lower/upper case when it applies, punctuation mark(s) if any. The whole output sentence must be typed in between quotes (").

In [221]:
i = input_sentences.index("Is anybody hurt?")
input_seq = encoder_input_sequences[i:i+1]
translation = translate_sentence(input_seq)
print('"' + translation + '"')

"quiconque est-il blessé ?"


## Question 3: Third translation example (validation sentence)
What is the translation produced by your encoder-decoder model for the following input sentence?

    "Does he live here?"

Your answer must respect the exact format of the output word sequence predicted by the model, including lower/upper case when it applies, punctuation mark(s) if any. The whole output sentence must be typed in between quotes (").

In [222]:
i = input_sentences.index("Does he live here?")
input_seq = encoder_input_sequences[i:i+1]
translation = translate_sentence(input_seq)
print('"' + translation + '"')

"est-ce que je dis ici ?"


## Question 4: Python code for translating an input sentence
Paste here the python code that you designed to produce the answers to the previous questions. It could typically include some import statements and, at least, one `translate` function taking as parameters the `input_string`, the `encoder` and the `decoder` models. Your function might have additional parameters as well. This function must return a string, which is the result you pasted to answer the 3 previous questions.

In [242]:
import os, sys
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

def translate(input_string, encoder, decoder, word2idx_inputs, word2idx_outputs, input_tokenizer, max_input_len, max_out_len):
    """
    Translate an English sentence into French.
    
    Parameters
    ----------
        input_string
            The sentence to translate.
        encoder
            The encoder model used.
        decoder
            The decoder model used.
        word2idx_inputs
            Word-to-index dictionary where input words are the keys and the corresponding integers are the values.
        word2idx_outputs
            Word-to-index dictionary where output words are the keys and the corresponding integers are the values.
        input_tokenizer
            Tokenizer used to transform the inputs into sequences of numbers.
        max_input_len
            Maximum length of the input sequences.
        max_out_len
            Maximum length of the output sequences.
    Returns
    -------
        output_string
            The translation of the input string found by the model.
    """
    
    idx2word_input = {v:k for k, v in word2idx_inputs.items()}
    idx2word_target = {v:k for k, v in word2idx_outputs.items()}

    def translate_sentence(input_seq):
        states_value = encoder.predict(input_seq)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = word2idx_outputs['<sos>']
        eos = word2idx_outputs['<eos>']
        output_sentence = []
        
        for _ in range(max_out_len):
            output_tokens, h, c = decoder.predict([target_seq] + states_value)
            idx = np.argmax(output_tokens[0, 0, :])
        
            if eos == idx:
                break
            
            word = ''
        
            if idx > 0:
                word = idx2word_target[idx]
                output_sentence.append(word)
                
            target_seq[0, 0] = idx
            states_value = [h, c]
        
        return ' '.join(output_sentence)
    
    return '"' + translate_sentence(pad_sequences(input_tokenizer.texts_to_sequences([input_string]), maxlen=max_input_len)) + '"'

# 3.4 Evaluation through the BLEU metric
The Bilingual Evaluation Understudy Score, or BLEU for short, is a metric for evaluating a generated sentence to a reference sentence.

A perfect match results in a score of 1.0, whereas a full mismatch results in a score of 0.0.

You will evaluate the validation data results of your seq2seq2 architecture according to the BLEU score.

## Question 1: Check the default BLEU Score computation
The Python Natural Language Toolkit library, or NLTK, provides an implementation of the BLEU score that you can use to evaluate your generated text against a reference.

The `sentence_bleu(...)` function evaluates a candidate sentence against one or more reference sentences. The reference sentences are provided as a list of sentences where each reference is a list of tokens. The candidate sentence is provided as a list of tokens. You can find additional information and the implementation of this function in the bleu documentation.

You should use the default parameters of the `sentence_bleu(...)` function, except that some n-gram counts may need to be smoothed. Use the smoothing `method1` in this regard.

For instance, with the following reference sentences and a single candidate sentence, what is the BLEU score obtained?
```python
reference = [['this', 'looks', 'highly', 'satisfactory', '<eos>'],['this', 'looks', 'good', 'indeed', '<eos>']]
candidate = ['this', 'is', 'very', 'good', 'indeed', '<eos>']
```

In [216]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
reference = [['this', 'looks', 'highly', 'satisfactory', '<eos>'], ['this', 'looks', 'good', 'indeed', '<eos>']]
candidate = ['this', 'is', 'very', 'good', 'indeed', '<eos>']
m1 = SmoothingFunction()
bleu = sentence_bleu(reference,
                     candidate,
                     smoothing_function=m1.method1)
print(bleu)

0.21711852081087685


## Question 2: BLEU Score on the validation data

What is the bleu score of your encoder-decoder prediction models evaluated on the validation data? It is assumed that you have been considering all the default hyper- parameters, summarized again below.
```python
NUM_SENTENCES = 20000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_SIZE = 50
BATCH_SIZE = 64
LSTM_NODES = 256
VALIDATION_SPLIT = 0.1
EPOCHS = 20
```
The above means that the validation data is made of the last 10% (2,000 sentences) of the originally loaded corpus. For this evaluation to make sense, the input sentences are supposed to be consistent with the format (tokenization, filtering of punctuations, ...) used when training your model. Nevertheless, the models are not supposed to have been fitted on the validation data, but just evaluated according to the accuracy metric. Here, you will consider only one reference for each translated sentence, since this is the information present in the original corpus. The reference sentences used to computed the BLEU score should also be consistent with the pre-processing (tokenization) of the output sequences, and the dictionary which has been built accordingly.

### Notes
1. Each reference sentence is assumed to end with a `<eos>` special token. The translated sentence is also assumed to end with this token (otherwise you would not know that the translation stops there!).

    The special token `<eos>` is considered since part of the prediction to be made is when the output sentence actually stops. The special token `<sos>` is not considered at the beginning of such reference sentence since it is trivial to predict that a sentence starts with this token.

2. You must compute the BLEU-2 score, which is not the default in the nltk implementation. In other words, the BLEU score is evaluated only one 1-grams and 2-grams since the corpus is made of very short sentences. You must use equal weights between 1-grams and 2-grams and the smoothing `method1`.

In [253]:
def bleu_score(encoder, decoder, word2idx_inputs, word2idx_outputs, input_tokenizer, output_tokenizer, input_sentences, output_sentences, max_input_len, max_out_len):
    """
    Compute the BLEU-2 score on the validation set.

    Parameters
    ----------
        encoder
            The encoder model used.
        decoder
            The decoder model used.
        word2idx_inputs
            Word-to-index dictionary where input words are the keys and the corresponding integers are the values.
        word2idx_outputs
            Word-to-index dictionary where output words are the keys and the corresponding integers are the values.
        input_tokenizer
            Tokenizer used to transform the inputs into sequences of numbers.
        output_tokenizer
            Tokenizer used to transform the outputs into sequences of numbers.
        input_sentences
            Sentences given as input to the models during training.
        output_sentences
            Sentences given as output to the models during training.
        max_input_len
            Maximum length of the input sequences.
        max_output_len
            Maximum length of the output sequences.
    Returns
    -------
        score
            The average BLEU-2 score on the validation set.
    """
    def translate(input_string, encoder, decoder, word2idx_inputs, word2idx_outputs, input_tokenizer, max_input_len, max_out_len):
        idx2word_input = {v:k for k, v in word2idx_inputs.items()}
        idx2word_target = {v:k for k, v in word2idx_outputs.items()}

        def translate_sentence(input_seq):
            states_value = encoder.predict(input_seq)
            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = word2idx_outputs['<sos>']
            eos = word2idx_outputs['<eos>']
            output_sentence = []

            for _ in range(max_out_len):
                output_tokens, h, c = decoder.predict([target_seq] + states_value)
                idx = np.argmax(output_tokens[0, 0, :])

                if eos == idx:
                    break

                word = ''

                if idx > 0:
                    word = idx2word_target[idx]
                    output_sentence.append(word)

                target_seq[0, 0] = idx
                states_value = [h, c]

            return ' '.join(output_sentence)

        return translate_sentence(pad_sequences(input_tokenizer.texts_to_sequences([input_string]), maxlen=max_input_len))
    
    total = 0
    weights = (1./2., 1./2.)
    for i, instring in enumerate(input_sentences[18000:]):
        candidate = output_tokenizer.texts_to_sequences([(translate(instring, encoder, decoder, word2idx_inputs, word2idx_outputs, input_tokenizer, max_input_len, max_out_len) + ' <eos>')])[0]
        reference = output_tokenizer.texts_to_sequences([output_sentences[18000 + i]])
        total += sentence_bleu(reference, candidate, smoothing_function=SmoothingFunction().method1, weights=weights)
    return total / 2000.


print(bleu_score(encoder_model, decoder_model, word2idx_inputs, word2idx_outputs, input_tokenizer, output_tokenizer, input_sentences, output_sentences, max_input_len, max_out_len))

0
[32, 27, 19, 9, 52, 4, 1]
[[19, 2634, 8926, 1]]
1
[32, 27, 3, 307, 144, 4, 1]
[[32, 309, 1546, 556, 1]]
2
[32, 27, 3, 307, 144, 4, 1]
[[32, 309, 912, 556, 1]]
3
[32, 27, 3, 307, 144, 4, 1]
[[8927, 144, 4, 1]]
4
[9, 192, 37, 4, 1]
[[44, 8928, 1]]
5
[9, 192, 37, 4, 1]
[[32, 904, 44, 2425, 4, 1]]
6
[32, 27, 29, 37, 146, 4, 1]
[[99, 8929, 347, 482, 4, 1]]
7
[32, 27, 29, 65, 4, 1]
[[18, 4451, 8930, 4, 1]]
8
[32, 27, 29, 65, 4, 1]
[[32, 27, 18, 4451, 4274, 4, 1]]
9
[26, 564, 35, 39, 332, 1]
[[26, 564, 35, 4098, 1]]
10
[11, 13, 126, 8, 5, 1]
[[11, 2830, 8, 20, 58, 1]]
11
[11, 13, 126, 8, 5, 1]
[[11, 2065, 8, 20, 134, 1]]
12
[11, 73, 8, 13, 48, 5, 1]
[[11, 83, 8, 8931, 1]]
13
[11, 73, 8, 91, 5, 1]
[[11, 44, 1493, 8, 23, 262, 1]]
14
[11, 73, 8, 91, 5, 1]
[[11, 83, 8, 432, 983, 1]]
15
[11, 73, 8, 91, 5, 1]
[[11, 73, 8, 432, 983, 1]]
16
[11, 13, 8, 13, 168, 5, 1]
[[11, 976, 8, 502, 1]]
17
[11, 13, 8, 13, 168, 5, 1]
[[11, 8932, 8, 502, 1]]
18
[11, 13, 126, 8, 5, 1]
[[11, 13, 1293, 41, 1]]
19
[11

[24, 96, 13, 160, 1]
[[725, 12, 79, 196, 5, 1]]
146
[516, 12, 43, 664, 5, 1]
[[2895, 12, 43, 779, 1]]
147
[65, 13, 271, 5, 1]
[[2499, 20, 143, 74, 271, 5, 1]]
148
[65, 13, 271, 5, 1]
[[1886, 20, 143, 74, 271, 5, 1]]
149
[65, 68, 5, 1]
[[1886, 359, 1]]
150
[725, 12, 51, 271, 5, 1]
[[516, 51, 271, 5, 1]]
151
[725, 12, 51, 271, 5, 1]
[[725, 51, 271, 5, 1]]
152
[65, 27, 19, 5, 1]
[[8960, 38, 13, 90, 5, 1]]
153
[265, 25, 382, 1]
[[312, 25, 4298, 1]]
154
[265, 25, 286, 12, 272, 1]
[[312, 1269, 814, 1]]
155
[265, 25, 286, 12, 272, 1]
[[265, 1269, 814, 1]]
156
[265, 25, 382, 1]
[[265, 17, 174, 12, 423, 1]]
157
[265, 18, 128, 20, 428, 1]
[[8961, 13, 8962, 5, 1]]
158
[312, 18, 43, 162, 5, 1]
[[265, 13, 931, 1]]
159
[312, 18, 43, 162, 5, 1]
[[312, 13, 931, 1]]
160
[312, 18, 312, 5, 1]
[[265, 4457, 1]]
161
[312, 18, 312, 5, 1]
[[312, 4457, 1]]
162
[312, 56, 694, 5, 1]
[[265, 113, 8963, 1]]
163
[312, 18, 678, 5, 1]
[[265, 18, 464, 1]]
164
[312, 56, 694, 5, 1]
[[265, 214, 1997, 5, 1]]
165
[312, 56, 

[7, 15, 16, 183, 203, 451, 1]
[[16, 17, 9029, 4248, 1]]
293
[16, 17, 414, 1]
[[16, 17, 9030, 414, 1]]
294
[16, 17, 5, 1]
[[16, 17, 9031, 1]]
295
[16, 17, 1]
[[7, 15, 3907, 1]]
296
[7, 15, 366, 7, 15, 366, 77, 67, 12, 136, 136, 157, 157, 1]
[[7, 15, 9032, 1]]
297
[16, 17, 120, 5, 1]
[[16, 17, 4195, 1]]
298
[16, 17, 120, 5, 1]
[[16, 17, 9033, 1]]
299
[7, 15, 195, 16, 5, 1]
[[16, 17, 4464, 1]]
300
[7, 15, 195, 16, 5, 1]
[[7, 15, 4464, 1]]
301
[16, 17, 585, 1]
[[16, 17, 436, 9034, 1]]
302
[7, 15, 448, 1]
[[16, 17, 4465, 1]]
303
[7, 15, 448, 1]
[[7, 15, 4465, 1]]
304
[7, 15, 38, 20, 234, 39, 334, 5, 1]
[[7, 364, 316, 18, 9035, 1]]
305
[7, 15, 195, 7, 223, 5, 1]
[[7, 15, 2313, 1]]
306
[16, 17, 28, 31, 23, 262, 1]
[[16, 17, 4432, 414, 1]]
307
[7, 15, 145, 13, 924, 1]
[[7, 15, 20, 179, 924, 1]]
308
[7, 15, 360, 265, 12, 377, 1]
[[7, 130, 24, 239, 20, 23, 2378, 1]]
309
[7, 15, 23, 67, 12, 236, 1]
[[16, 17, 2096, 199, 709, 1]]
310
[7, 15, 273, 1]
[[7, 60, 37, 273, 1]]
311
[7, 15, 23, 67, 12, 39,

[183, 5, 1]
[[7, 193, 9085, 1]]
440
[7, 15, 17, 174, 340, 1]
[[16, 17, 381, 2240, 1]]
441
[7, 15, 448, 1]
[[7, 15, 4475, 1]]
442
[7, 15, 448, 1]
[[16, 17, 4475, 1]]
443
[16, 17, 67, 12, 12, 225, 1]
[[16, 17, 1852, 12, 1787, 1]]
444
[7, 15, 38, 20, 37, 1023, 12, 234, 315, 53, 1]
[[7, 15, 351, 655, 1]]
445
[7, 15, 38, 20, 37, 1023, 12, 234, 315, 53, 1]
[[7, 193, 15, 351, 1098, 1]]
446
[7, 15, 17, 174, 12, 37, 1]
[[16, 17, 2297, 12, 18, 575, 1]]
447
[16, 17, 349, 12, 18, 72, 1]
[[16, 17, 2653, 12, 957, 9086, 1]]
448
[7, 15, 23, 80, 1]
[[7, 15, 9087, 3860, 9088, 1]]
449
[7, 15, 23, 67, 12, 10, 12, 18, 4, 1]
[[7, 15, 9089, 12, 9090, 1]]
450
[7, 15, 23, 67, 12, 18, 359, 1]
[[7, 15, 96, 13, 1471, 1]]
451
[7, 15, 23, 67, 12, 18, 359, 1]
[[7, 15, 96, 18, 9091, 1]]
452
[7, 15, 38, 20, 37, 1]
[[7, 349, 979, 1]]
453
[7, 15, 174, 340, 1]
[[7, 15, 187, 2100, 1]]
454
[7, 15, 174, 340, 1]
[[7, 15, 9092, 96, 179, 1441, 1]]
455
[7, 15, 23, 67, 44, 171, 171, 171, 441, 1]
[[7, 24, 64, 85, 1]]
456
[7, 42, 

[3, 6, 17, 24, 12, 1]
[[3, 6, 2483, 1]]
593
[3, 11, 6, 8, 23, 106, 1]
[[14, 1031, 510, 106, 1]]
594
[3, 11, 6, 8, 23, 106, 1]
[[14, 1031, 12, 357, 613, 1]]
595
[3, 69, 51, 136, 136, 136, 349, 4, 1]
[[3, 2863, 51, 1317, 1]]
596
[3, 69, 51, 136, 136, 136, 349, 4, 1]
[[3, 6, 23, 67, 12, 9137, 51, 1317, 1]]
597
[3, 6, 20, 18, 129, 1]
[[3, 6, 96, 18, 129, 1]]
598
[3, 6, 96, 56, 433, 428, 1]
[[3, 6, 56, 9138, 1]]
599
[3, 6, 96, 56, 433, 428, 1]
[[3, 6, 56, 9139, 1]]
600
[3, 11, 6, 8, 20, 131, 1]
[[3, 11, 6, 8, 101, 114, 1]]
601
[3, 11, 6, 8, 20, 131, 1]
[[3, 11, 6, 8, 101, 131, 1]]
602
[3, 6, 159, 165, 1]
[[3, 6, 187, 389, 1]]
603
[3, 6, 125, 537, 1]
[[3, 6, 91, 9140, 5, 1]]
604
[3, 6, 125, 537, 1]
[[3, 6, 91, 9141, 5, 1]]
605
[3, 6, 23, 67, 12, 55, 218, 1]
[[3, 6, 23, 67, 1415, 1]]
606
[3, 6, 96, 26, 479, 26, 334, 138, 1]
[[14, 13, 435, 627, 1]]
607
[3, 6, 96, 26, 479, 26, 334, 138, 1]
[[3, 6, 51, 435, 627, 1]]
608
[3, 6, 53, 322, 1]
[[3, 6, 53, 865, 1]]
609
[3, 6, 53, 668, 1]
[[3, 6, 53, 1

...

[3, 257, 27, 16, 27, 20, 136, 21, 57, 17, 66, 67, 12, 1]
[[3, 257, 27, 14, 283, 1]]
1320
[3, 13, 257, 27, 170, 94, 164, 1]
[[3, 257, 309, 94, 185, 1]]
1321
[3, 257, 27, 29, 65, 85, 1]
[[3, 257, 27, 16, 172, 1]]
1322
[3, 257, 27, 16, 172, 1]
[[3, 257, 27, 16, 172, 1]]
1323
[3, 257, 27, 16, 172, 1]
[[3, 257, 27, 16, 85, 1]]
1324
[3, 257, 27, 14, 13, 115, 295, 1]
[[3, 257, 27, 16, 282, 1]]
1325
[3, 257, 27, 14, 13, 115, 295, 1]
[[3, 257, 27, 16, 194, 1]]
1326
[3, 257, 27, 16, 172, 1]
[[3, 652, 27, 16, 368, 1]]
1327
[3, 257, 146, 33, 295, 1]
[[3, 257, 9318, 1]]
1328
[3, 13, 257, 27, 14, 120, 12, 134, 1]
[[9319, 1]]
1329
[3, 13, 257, 27, 14, 120, 12, 134, 1]
[[9320, 1]]
1330
[3, 93, 8, 48, 93, 20, 18, 87, 158, 908, 67, 12, 60, 1]
[[14, 182, 9321, 9322, 1]]
1331
[14, 37, 25, 380, 1]
[[14, 182, 17, 2591, 1]]
1332
[14, 182, 25, 37, 5, 1]
[[14, 182, 25, 2686, 12, 3644, 1]]
1333
[14, 666, 89, 191, 1]
[[3, 105, 6, 197, 1]]
1334
[14, 756, 20, 25, 19, 5, 1]
[[14, 1151, 12, 4221, 1]]
1335
[3, 21, 81

[3, 11, 21, 6, 8, 41, 1]
[[9366, 1]]
1463
[3, 11, 21, 6, 8, 41, 1]
[[9367, 1]]
1464
[3, 11, 133, 8, 134, 1]
[[3, 11, 13, 9368, 41, 1]]
1465
[3, 11, 9, 757, 41, 1]
[[3, 11, 21, 9369, 8, 111, 114, 1]]
1466
[3, 11, 6, 8, 125, 172, 1]
[[3, 4253, 8, 50, 12, 272, 1]]
1467
[3, 11, 6, 8, 295, 1]
[[3, 11, 9370, 41, 1]]
1468
[3, 11, 6, 8, 207, 1]
[[3, 11, 9371, 8, 167, 1]]
1469
[14, 184, 54, 1]
[[3, 3345, 111, 54, 1]]
1470
[3, 57, 12, 13, 443, 1]
[[3, 21, 69, 51, 9372, 20, 179, 1933, 1]]
1471
[14, 756, 20, 37, 5, 1]
[[1539, 1605, 1]]
1472
[14, 475, 30, 163, 1]
[[14, 475, 30, 163, 1]]
1473
[3, 66, 475, 20, 54, 1]
[[14, 475, 30, 163, 1]]
1474
[14, 756, 20, 58, 1]
[[14, 3899, 20, 377, 1]]
1475
[13, 383, 1]
[[3, 9373, 1]]
1476
[82, 60, 64, 78, 302, 1]
[[3, 492, 135, 432, 23, 262, 1]]
1477
[3, 6, 154, 1]
[[3, 492, 2478, 1]]
1478
[3, 6, 154, 1]
[[3, 492, 2479, 1]]
1479
[3, 104, 138, 1]
[[3, 1800, 706, 510, 143, 138, 1]]
1480
[477, 25, 382, 1]
[[477, 17, 549, 1]]
1481
[477, 26, 284, 5, 1]
[[477, 535, 1

[3, 6, 53, 165, 1]
[[3, 6, 89, 1076, 347, 165, 1]]
1616
[3, 6, 23, 67, 12, 224, 1]
[[3, 601, 224, 1]]
1617
[3, 6, 20, 26, 77, 183, 257, 93, 27, 18, 98, 208, 398, 1]
[[3, 1368, 51, 4266, 1]]
1618
[3, 6, 20, 26, 77, 183, 257, 93, 27, 18, 98, 208, 398, 1]
[[3, 9408, 1]]
1619
[3, 11, 6, 8, 17, 747, 1]
[[3, 11, 6, 8, 1856, 1]]
1620
[3, 11, 6, 8, 17, 747, 1]
[[3, 11, 6, 8, 9409, 1]]
1621
[3, 11, 6, 8, 17, 617, 1]
[[3, 11, 6, 8, 17, 1995, 1]]
1622
[3, 11, 6, 8, 17, 34, 12, 18, 77, 58, 1]
[[3, 11, 9410, 8, 18, 9411, 1]]
1623
[3, 11, 6, 8, 17, 34, 12, 18, 77, 58, 1]
[[3, 11, 6, 8, 9412, 1]]
1624
[3, 11, 6, 8, 17, 34, 12, 18, 77, 58, 1]
[[3, 11, 6, 8, 9413, 1]]
1625
[3, 11, 6, 8, 17, 250, 1]
[[3, 11, 6, 8, 3359, 1]]
1626
[3, 11, 6, 8, 17, 34, 20, 18, 129, 1]
[[3, 11, 6, 8, 3828, 1]]
1627
[3, 11, 6, 8, 17, 34, 20, 18, 129, 1]
[[3, 11, 6, 8, 3829, 1]]
1628
[3, 11, 6, 8, 17, 34, 20, 18, 129, 1]
[[3, 11, 6, 8, 9414, 1]]
1629
[3, 11, 6, 8, 17, 163, 1]
[[3, 11, 6, 8, 17, 3382, 1]]
1630
[3, 11, 6, 8, 1

[32, 38, 29, 4, 1]
[[32, 1521, 1081, 4, 1]]
1756
[32, 187, 4, 1]
[[32, 27, 29, 976, 4, 1]]
1757
[32, 27, 16, 452, 4, 1]
[[32, 9440, 4, 1]]
1758
[32, 27, 29, 15, 23, 67, 12, 4, 1]
[[32, 4350, 1]]
1759
[32, 27, 16, 8, 4, 1]
[[352, 8, 1081, 4, 1]]
1760
[32, 39, 1335, 4, 1]
[[32, 9441, 4, 1]]
1761
[32, 26, 398, 4, 1]
[[32, 12, 4542, 4543, 1]]
1762
[32, 187, 4, 1]
[[32, 9442, 4, 1]]
1763
[16, 71, 23, 67, 4, 1]
[[32, 27, 16, 71, 434, 4, 1]]
1764
[32, 27, 16, 113, 714, 1]
[[32, 99, 56, 1775, 4, 1]]
1765
[32, 27, 16, 56, 4, 1]
[[32, 27, 16, 71, 902, 4, 1]]
1766
[32, 113, 714, 1]
[[32, 113, 2706, 1]]
1767
[32, 27, 16, 113, 714, 1]
[[32, 113, 507, 4, 1]]
1768
[32, 27, 16, 113, 714, 1]
[[32, 56, 507, 4, 1]]
1769
[18, 33, 51, 387, 4, 1]
[[13, 4487, 107, 2288, 1]]
1770
[32, 27, 16, 18, 270, 4, 1]
[[32, 27, 13, 1982, 15, 9443, 1]]
1771
[18, 33, 51, 25, 547, 20, 18, 55, 1]
[[32, 27, 13, 1533, 15, 2808, 1]]
1772
[18, 196, 15, 23, 18, 196, 4, 1]
[[32, 27, 18, 4321, 15, 9444, 1]]
1773
[13, 60, 280, 12, 

[16, 25, 171, 354, 1]
[[16, 25, 823, 985, 1]]
1903
[16, 17, 345, 1]
[[16, 4285, 101, 1077, 1]]
1904
[16, 17, 345, 1]
[[16, 9492, 12, 2855, 1]]
1905
[16, 17, 345, 1]
[[7, 130, 24, 8, 20, 9493, 1]]
1906
[16, 17, 345, 1]
[[7, 130, 24, 281, 9494, 1]]
1907
[16, 17, 345, 1]
[[3, 11, 261, 8, 102, 15, 13, 374, 1]]
1908
[16, 17, 345, 1]
[[7, 130, 24, 8, 12, 190, 60, 68, 39, 9495, 74, 9496, 1]]
1909
[16, 25, 196, 5, 1]
[[16, 17, 9497, 9498, 1]]
1910
[16, 17, 268, 1]
[[16, 25, 9499, 431, 1]]
1911
[16, 17, 163, 1]
[[16, 25, 1831, 2598, 1]]
1912
[16, 30, 27, 3, 3, 59, 49, 106, 1]
[[16, 38, 30, 27, 3, 59, 164, 1]]
1913
[7, 15, 38, 20, 37, 79, 129, 1]
[[16, 38, 30, 27, 9500, 1]]
1914
[16, 708, 133, 133, 133, 39, 478, 21, 85, 1]
[[16, 708, 9501, 1]]
1915
[16, 708, 133, 133, 133, 39, 478, 21, 85, 1]
[[239, 127, 12, 1433, 1]]
1916
[16, 708, 133, 133, 133, 39, 478, 21, 85, 1]
[[239, 12, 29, 127, 12, 1433, 1]]
1917
[16, 38, 20, 55, 50, 27, 30, 49, 24, 4, 1]
[[590, 38, 15, 282, 1]]
1918
[16, 38, 13, 20, 7,

## Question 3: Python code for BLEU score computation

Submit here the python code that you designed to produce the answers to the previous questions.

In [ ]:
import os, sys
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def bleu_score(encoder, decoder, word2idx_inputs, word2idx_outputs, input_tokenizer, output_tokenizer, input_sentences, output_sentences, max_input_len, max_out_len):
    """
    Compute the BLEU-2 score on the validation set.

    Parameters
    ----------
        encoder
            The encoder model used.
        decoder
            The decoder model used.
        word2idx_inputs
            Word-to-index dictionary where input words are the keys and the corresponding integers are the values.
        word2idx_outputs
            Word-to-index dictionary where output words are the keys and the corresponding integers are the values.
        input_tokenizer
            Tokenizer used to transform the inputs into sequences of numbers.
        output_tokenizer
            Tokenizer used to transform the outputs into sequences of numbers.
        input_sentences
            Sentences given as input to the models during training.
        output_sentences
            Sentences given as output to the models during training.
        max_input_len
            Maximum length of the input sequences.
        max_output_len
            Maximum length of the output sequences.
    Returns
    -------
        score
            The average BLEU-2 score on the validation set.
    """
    def translate(input_string, encoder, decoder, word2idx_inputs, word2idx_outputs, input_tokenizer, max_input_len, max_out_len):
        idx2word_input = {v:k for k, v in word2idx_inputs.items()}
        idx2word_target = {v:k for k, v in word2idx_outputs.items()}

        def translate_sentence(input_seq):
            states_value = encoder.predict(input_seq)
            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = word2idx_outputs['<sos>']
            eos = word2idx_outputs['<eos>']
            output_sentence = []

            for _ in range(max_out_len):
                output_tokens, h, c = decoder.predict([target_seq] + states_value)
                idx = np.argmax(output_tokens[0, 0, :])

                if eos == idx:
                    break

                word = ''

                if idx > 0:
                    word = idx2word_target[idx]
                    output_sentence.append(word)

                target_seq[0, 0] = idx
                states_value = [h, c]

            return ' '.join(output_sentence)

        return translate_sentence(pad_sequences(input_tokenizer.texts_to_sequences([input_string]), maxlen=max_input_len))
    
    total = 0
    weights = (1./2., 1./2.)
    for i, instring in enumerate(input_sentences[18000:]):
        candidate = output_tokenizer.texts_to_sequences([(translate(instring, encoder, decoder, word2idx_inputs, word2idx_outputs, input_tokenizer, max_input_len, max_out_len) + ' <eos>')])[0]
        reference = output_tokenizer.texts_to_sequences([output_sentences[18000 + i]])
        total += sentence_bleu(reference, candidate, smoothing_function=SmoothingFunction().method1, weights=weights)
    return total / 2000.